In [12]:
from preprocess import StandardizeTransform
from models.en_denoiser import EnDenoiser
from moleculib.protein.dataset import ProteinDataset
from moleculib.protein.batch import PadBatch
from torch.utils.data import DataLoader
from models.en_denoiser import EnDenoiser
from visualize import preds_to_pdb
from tmtools import tm_align
import numpy as np
import torch

In [2]:
DATA_DIR = "../data/single/"
transform = StandardizeTransform()
dataset = ProteinDataset(DATA_DIR, transform=[transform], preload=True)
dataloader = DataLoader(dataset, collate_fn=PadBatch.collate, batch_size=2)

/Users/ido/genomator/venv/lib/python3.9/site-packages/biotite/structure/io/pdb/file.py:423: UserWarning: 1368 elements were guessed from atom_name.
  warn("{} elements were guessed from atom_name.".format(rep_num))


In [3]:
batch = next(iter(dataloader))

In [4]:
model = EnDenoiser()

In [5]:
c, s, m = model.prepare_inputs(batch)

In [6]:
crds = [c[0], 1 + c[1]]
seq = str(batch.sequence[0][:model.trim])
pdb_fname = "align.pdb"
bb_start = 1
bb_end = 2

In [11]:
preds_to_pdb(crds, seq, pdb_fname, bb_start, bb_end, align=False)

File align.pdb has been saved.


In [36]:
ref = torch.load("../ref.pt", map_location="cpu")[0].numpy()

In [37]:
pred = torch.load("../pred_50700.pt", map_location="cpu")[0].numpy()

In [38]:
crds = [ref, pred]

In [39]:
preds_to_pdb(crds, seq, pdb_fname, bb_start, bb_end, align=False)

File align.pdb has been saved.


In [40]:
a = tm_align(ref, pred, seq, seq)

In [41]:
a.tm_norm_chain2

0.9998702904400399

In [42]:
pred

array([[ 0.93726848,  0.97324451,  1.33078328],
       [ 0.60194966,  0.72120939,  1.22878964],
       [ 0.32096997,  1.01054667,  1.10931015],
       [-0.1099875 ,  0.99000839,  1.03395125],
       [-0.3348327 ,  1.26024304,  0.77284863],
       [-0.56380985,  1.522899  ,  0.98346683],
       [-0.9666418 ,  1.40965548,  0.96220666],
       [-1.16815634,  1.63987147,  0.68294588],
       [-0.81589207,  1.77038535,  0.47852093],
       [-0.81174086,  1.64502959,  0.07206279],
       [-0.5022805 ,  1.40538124, -0.09652186],
       [-0.22940655,  1.66804145, -0.27622512],
       [-0.26871472,  1.99312175, -0.01368144],
       [ 0.04482272,  1.89732952,  0.21899612],
       [-0.13195122,  1.87902268,  0.65738279],
       [ 0.01565692,  1.57939372,  0.91924104],
       [ 0.05461532,  1.51254907,  1.32834919],
       [ 0.33214772,  1.23413513,  1.54860238],
       [ 0.21058079,  0.89618461,  1.64650984],
       [ 0.32888534,  0.56035603,  1.8523658 ],
       [ 0.47108217,  0.17346773,  1.640

In [43]:
a.u

array([[ 0.03128684,  0.80368984,  0.59422535],
       [ 0.85638451, -0.32811262,  0.39868243],
       [ 0.51538986,  0.49641187, -0.69853314]])

In [44]:
a.t

array([0.10117608, 0.02881313, 0.00105552])

In [54]:
np.matmul(pred, a.u) + a.t

array([[ 1.64984398e+00,  1.12336910e+00,  1.64234868e-02],
       [ 1.37094745e+00,  8.85941830e-01, -2.12067496e-01],
       [ 1.54836193e+00,  5.05875054e-01, -1.80218678e-01],
       [ 1.47845075e+00,  1.28848734e-01, -3.91852100e-01],
       [ 1.56827118e+00, -2.70138907e-01, -2.35334175e-01],
       [ 1.89459219e+00, -4.35792879e-01, -4.13805676e-01],
       [ 1.77405159e+00, -7.32941994e-01, -6.83475902e-01],
       [ 1.82097206e+00, -1.10906233e+00, -5.16364967e-01],
       [ 1.83840481e+00, -9.70253335e-01, -1.12209422e-01],
       [ 1.52169756e+00, -1.12755689e+00,  1.24204678e-01],
       [ 1.23926164e+00, -8.83902522e-01,  3.30312244e-01],
       [ 1.38011990e+00, -8.39985455e-01,  7.22707559e-01],
       [ 1.79259615e+00, -8.47910184e-01,  6.45557992e-01],
       [ 1.84029042e+00, -4.48988781e-01,  6.31146226e-01],
       [ 2.04502208e+00, -3.67433149e-01,  2.12576430e-01],
       [ 1.92800176e+00, -2.05003969e-02, -2.08453893e-03],
       [ 2.08282611e+00,  2.35828784e-01

In [48]:
ref

array([[ 1.53203953,  1.04852751, -0.08212866],
       [ 1.27737373,  0.77263843, -0.29612865],
       [ 1.43659581,  0.38419374, -0.26257312],
       [ 1.38648425,  0.02863852, -0.48112866],
       [ 1.46615038, -0.36525016, -0.34623976],
       [ 1.8197067 , -0.53147263, -0.52535088],
       [ 1.68903891, -0.83825058, -0.79935088],
       [ 1.74237346, -1.22869587, -0.64335088],
       [ 1.75170675, -1.10802937, -0.23479534],
       [ 1.4449288 , -1.25347275,  0.02964912],
       [ 1.15259583, -0.99325106,  0.20753799],
       [ 1.29470602, -0.95269489,  0.61087139],
       [ 1.70581679, -0.97725137,  0.51176025],
       [ 1.77804003, -0.55247381,  0.48698242],
       [ 1.95303969, -0.49036206,  0.0986491 ],
       [ 1.84237278, -0.13247225, -0.11290644],
       [ 1.98626147,  0.12441624, -0.41801755],
       [ 1.89437304,  0.53808202, -0.45979532],
       [ 1.62137296,  0.64597183, -0.76679532],
       [ 1.4430401 ,  0.97297107, -0.95079534],
       [ 1.04070652,  1.09586027, -0.873